# Preamble

In [1]:
from sympy import *
init_printing(use_latex='mathjax')

# Plotting
%matplotlib inline
## Make inline plots raster graphics
from IPython.display import set_matplotlib_formats
## Import modules for plotting and data analysis
import matplotlib.pyplot as plt
from matplotlib import gridspec,rc,colors
import matplotlib.ticker as plticker
## Parameters for seaborn plots
import seaborn as sns
sns.set(style='white',font_scale=1.25,
        rc={"xtick.major.size": 6, "ytick.major.size": 6,
            'text.usetex': False, 'font.family': 'serif', 'font.serif': ['Times']})

import pandas as pd
pd.set_option('mode.chained_assignment',None)

import numpy as np
from scipy.optimize import fsolve, root
from scipy.integrate import ode
backend = 'dopri5'
import warnings

# Timer
import time

import csv
from numpy.linalg import norm

from itertools import cycle
palette_size = 10;
clrs = sns.color_palette("Reds",palette_size)
iclrs = cycle(clrs) # iterated colors

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Main module

## Baseline parameter values

In [7]:
baseline = {'μ': 1/28., 'μbar': 1/60., 'd': 0.13, 'c': .04, 'α': 0.3, 'θ': 0.45, 'κ': 40., 'L': .2}
tmx = 720.

## Cost of resistance

In [3]:
# Poisson brackets
PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)

In [ ]:
File = open("../results/sensitivity_c.csv", 'w')
File.write("outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x\n")
writer = csv.writer(File,lineterminator='\n')

step = 0.01
sets = [np.arange(baseline['c'],-step,-step),np.arange(baseline['c']+step,0.1+step,step)]
for set_ in sets:
    end_switching_curve = {'t': 23.36, 'x': .9292}
    end_switching_curve_prev_t = end_switching_curve['t']
    tuniv = tmx-30.
    for c in set_:
        print("Parameters: c = %.3f"%c)
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; α = baseline['α']; θ = baseline['θ']; κ = baseline['κ']; L = baseline['L']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)*(1-L); Θ = θ+σ*(1-θ)*L
        Eminus = (α*A-Θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-Θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-6):
            end_switching_curve = {'t': 2*end_switching_curve['t']-end_switching_curve_prev_t-.001, 
                                   'x': end_switching_curve['x']-0.002} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-6):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [c,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

## Alpha

In [ ]:
File = open("../results/sensitivity_alpha.csv", 'w')
File.write("variable,value,outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x\n")
writer = csv.writer(File,lineterminator='\n')

step = 0.001
sets = [np.arange(baseline['α']+step,baseline['θ']+step,step),np.arange(baseline['α'],-step,-step)]
# sets = [np.arange(.199,-step,-step)]
for set_ in sets:
#     end_switching_curve = {'t': 23.36, 'x': .9592}
    tuniv = tmx-30.
    end_switching_curve = {'t': 21.4,'x': .84}
    end_switching_curve_prev_t = end_switching_curve['t']
    for α in set_:    
        print("Parameters: α = %.3f"%α)
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; c = baseline['c']; θ = baseline['θ']; κ = baseline['κ']; L = baseline['L']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)*(1-L); Θ = θ+σ*(1-θ)*L
        Eminus = (α*A-Θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-Θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-9; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-5):
            end_switching_curve = {'t': 2*end_switching_curve['t']-end_switching_curve_prev_t-.002, 
                                   'x': end_switching_curve['x']+0.01} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-6):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [α,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

## Theta

In [152]:
File = open("../results/sensitivity_theta.csv", 'w')
File.write("variable,value,outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x\n")
writer = csv.writer(File,lineterminator='\n')

step = 0.0025
sets = [np.arange(.224,-step,-step)] #np.arange(baseline['θ'],1.+step,step),
# sets = [np.arange(.199,-step,-step)]
for set_ in sets:
    end_switching_curve = {'t': 30.2, 'x': .8874}
    tuniv = tmx-30.
    end_switching_curve_prev_t = end_switching_curve['t']
    for θ in set_:    
        print("Parameters: θ = %.4f"%θ)
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; c = baseline['c']; α = baseline['α']; κ = baseline['κ']; L = baseline['L']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)*(1-L); Θ = θ+σ*(1-θ)*L
        Eminus = (α*A-Θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-Θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-9; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-5):
            end_switching_curve = {'t': 2*end_switching_curve['t']-end_switching_curve_prev_t-.001, 
                                   'x': end_switching_curve['x']*1.01} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-5):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [θ,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

Parameters: θ = 0.2240


KeyboardInterrupt: 

## kappa

In [182]:
File = open("../results/sensitivity_kappa.csv", 'w')
File.write("kappa,value,outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x\n")
writer = csv.writer(File,lineterminator='\n')

step = .5
sets = [np.arange(120,100+step,step),np.arange(17.5,-step,-step)]
# sets = [np.arange(baseline['κ'],100+step,step),np.arange(baseline['κ'],-step,-step)]
# sets = [np.arange(.199,-step,-step)]
for set_ in sets:
#     end_switching_curve = {'t': 30.2, 'x': .8874}
    end_switching_curve = {'t': 19.2, 'x': .974}
    tuniv = tmx-30.
    end_switching_curve_prev_t = end_switching_curve['t']
    for κ in set_:    
        print("Parameters: κ = %.4f"%κ)
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; c = baseline['c']; α = baseline['α']; L = baseline['L']; θ = baseline['θ']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)*(1-L); Θ = θ+σ*(1-θ)*L
        Eminus = (α*A-Θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-Θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-9; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-6):
            end_switching_curve = {'t': 2*end_switching_curve['t']-.01-end_switching_curve_prev_t,
                                   'x': end_switching_curve['x']*1.05} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-6):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [κ,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

Parameters: κ = 17.5000
Ending point: t = 32.51, x = 88.23%  Checking the solution: (4.8900873246006914e-13, 5.4438016270079181e-14)
tuniv = 679.53  Checking the solution:  -2.96984659087e-15
Fold-change in tumor size: 10.19
[17.5, 10.1858959149519, 32.512652265964199, 0.88230938778640555, 679.5333242065673, 0.72450406746881291, 4.8900873246006914e-13, 5.4438016270079181e-14, -2.9698465908722937e-15]
Parameters: κ = 17.0000
Ending point: t = 33.49, x = 88.04%  Checking the solution: (-1.4185440369288603e-08, -4.6178737433034508e-08)
tuniv = 679.35  Checking the solution:  4.31307767279e-13
Fold-change in tumor size: 12.06
[17.0, 12.0584163930790, 33.494250031268777, 0.88043011676305605, 679.353821564903, 0.72191993587870651, -1.4185440369288603e-08, -4.6178737433034508e-08, 4.313077672790655e-13]
Parameters: κ = 16.5000
Ending point: t = 34.66, x = 87.85%  Checking the solution: (-2.9788388986001917e-08, -2.4437456724530445e-07)
tuniv = 681.34  Checking the solution:  4.91127971625e-12

KeyboardInterrupt: 

In [183]:
File.close()

## L

In [ ]:
File = open("../results/sensitivity_L.csv", 'w')
File.write("L,value,outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x\n")
writer = csv.writer(File,lineterminator='\n')

step = 0.05
sets = [np.arange(.99,1.+step,step)]#,np.arange(baseline['L'],-step,-step),
# sets = [np.arange(.199,-step,-step)]
for set_ in sets:
#     end_switching_curve = {'t': 30.2, 'x': .8874}
    end_switching_curve = {'t': 19.2, 'x': .974}
    tuniv = tmx-30.
    end_switching_curve_prev_t = end_switching_curve['t']
    for L in set_:    
        print("Parameters: L = %.4f"%L)
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; c = baseline['c']; α = baseline['α']; κ = baseline['κ']; θ = baseline['θ']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)*(1-L); Θ = θ+σ*(1-θ)*L
        Eminus = (α*A-Θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-Θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-9; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-6):
            end_switching_curve = {'t': end_switching_curve['t']-.01, #-end_switching_curve_prev_t 
                                   'x': end_switching_curve['x']*.95} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-6):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [L,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

In [164]:
File.close()